In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
#载入并分析数据
(x_train,y_train),(x_val,y_val)=keras.datasets.cifar10.load_data()
print('训练样本维度：',x_train.shape)
print('样本最小值：',x_train.min())
print('样本最大值：',x_train.max())
print('标签维度：',y_train.shape)
print('标签处理后维度：',tf.squeeze(y_train).shape)

训练样本维度： (50000, 32, 32, 3)
样本最小值： 0
样本最大值： 255
标签维度： (50000, 1)
标签处理后维度： (50000,)


In [13]:
#预处理
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.-0.5
    x = tf.reshape(x,[-1,32*32*3])
    y = tf.squeeze(y)
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=10)
    return x,y

In [14]:
#准备喂入网络的数据batch
db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(200).map(preprocess).shuffle(10000)
db_val = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(200).map(preprocess).shuffle(10000)

#测试batch
db_iter = iter(db_train)
sample = next(db_iter)
print(sample[0].shape)
print(sample[1].shape)

(200, 3072)
(200, 10)


In [15]:
#自定义层
class MyDense(keras.layers.Layer):
    def __init__(self,in_dim,out_dim):
        super(MyDense,self).__init__()
        self.kernel = self.add_variable('w',[in_dim,out_dim])
        self.bias = self.add_variable('b',[out_dim])
        
    def call(self,inputs):
        x = inputs@self.kernel+self.bias
        return x

In [16]:
#自定义网络
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.l1 = MyDense(32*32*3,256)
        self.l2 = MyDense(256,128)
        self.l3 = MyDense(128,64)
        self.l4 = MyDense(64,32)
        self.l5 = MyDense(32,10)
        
    def call(self,inputs):
        out = self.l1(inputs)
        out = tf.nn.relu(out)
        out = self.l2(out)
        out = tf.nn.relu(out)
        out = self.l3(out)
        out = tf.nn.relu(out)
        out = self.l4(out)
        out = tf.nn.relu(out)
        out = self.l5(out)
        out = tf.nn.softmax(out)
        return out

In [17]:
my_model = MyModel()

In [19]:
my_model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.1),loss = tf.losses.categorical_crossentropy,metrics=['accuracy'])
my_model.fit(db_train,epochs=30,validation_data=db_val,validation_freq=2)

Epoch 1/30
250/250 [==============================] - 2s 9ms/step - loss: 1.0066 - accuracy: 0.6441
Epoch 2/30
250/250 [==============================] - 2s 10ms/step - loss: 0.9653 - accuracy: 0.6584 - val_loss: 1.4622 - val_accuracy: 0.5349
Epoch 3/30
250/250 [==============================] - 2s 8ms/step - loss: 0.9314 - accuracy: 0.6724
Epoch 4/30
250/250 [==============================] - 2s 9ms/step - loss: 0.8913 - accuracy: 0.6856 - val_loss: 1.4842 - val_accuracy: 0.5314
Epoch 5/30
250/250 [==============================] - 2s 8ms/step - loss: 0.8520 - accuracy: 0.7016
Epoch 6/30
250/250 [==============================] - 2s 9ms/step - loss: 0.8268 - accuracy: 0.7112 - val_loss: 1.6752 - val_accuracy: 0.4927
Epoch 7/30
250/250 [==============================] - 2s 8ms/step - loss: 0.7996 - accuracy: 0.7200
Epoch 8/30
250/250 [==============================] - 2s 9ms/step - loss: 0.7522 - accuracy: 0.7372 - val_loss: 1.6852 - val_accuracy: 0.5059
Epoch 9/30
250/250 [===========

In [20]:
my_model.evaluate(db_val)#过拟合比较严正

50/50 [==============================] - 0s 6ms/step - loss: 2.8110 - accuracy: 0.5028


[2.810979652404785, 0.5028]